In [1]:
from MA4Strategy import generate_candlestick_with_emas, load_data, evaluate_model
from MA4Strategy import build_cnn_model, build_cnn_lstm_model, build_dense_model, plot_accuracy, cal_accuracy
import pandas as pd
import tensorflow as tf
import os
import pickle
import gc

2025-06-21 10:07:24.841701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-21 10:07:24.852765: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-21 10:07:24.855966: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-21 10:07:24.864983: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-21 10:07:25.409673: W tensorflow/com

In [2]:
result_file = "4MAs_result.csv"

In [3]:
def main(ticker, window_days, alpha, epochs, predict_days, batch_size, result_df):
    # if model file exist, skip the process
    model_file = os.path.join("models", f"{ticker}_{window_days}_{alpha}_{epochs}_{predict_days}_{batch_size}.pkl")
    if os.path.exists(model_file):
        print(f"{model_file} model exist")
        return result_df

    # Generate and split data
    train_p = f"{ticker}_{window_days}_{alpha}_{epochs}_{predict_days}_{batch_size}"
    test_p = f"{ticker}_{window_days}_{alpha}_{epochs}_{predict_days}_{batch_size}"
    train_charts, train_labels, test_charts, test_labels, fulldata = generate_candlestick_with_emas(ticker, predict_days, window_days, alpha,
                                                                                                   train_p, test_p)

    if not train_charts or not test_charts:
        print("No charts generated, exiting.")
        return result_df
        
    em_y = fulldata['EMStrend'].dropna()
    print(len(em_y))
    lb_y = fulldata['Labels'].dropna()
    print(len(lb_y))

    # Load training and testing data
    X_train, y_train = load_data(os.path.join("train_metadata", f"{train_p}_metadata.csv"))
    X_test, y_test = load_data(os.path.join("test_metadata", f"{test_p}_metadata.csv"))
    
    # print("Trainning data:  ", X_train, ",", y_train)
    # print("Testing data:  ", X_test, ",", y_test)
    
    if X_train.size == 0 or X_test.size == 0:
        print("No valid images processed, exiting.")
        return result_df
        
    # Define models to train
    models = [build_cnn_model(), build_cnn_lstm_model(), build_dense_model()]
    
    # Train and evaluate each model
    for model, model_name in models:
        print(f"\nTraining {model_name}...")
        history = model.fit(X_train, y_train, epochs=epochs, validation_split=0.2, batch_size=batch_size, verbose=1)
        plot_accuracy(ticker, history, model_name)
        acc, prec, recall, f1 = evaluate_model(ticker, model, X_test, y_test, model_name)
        # Append the row
        new_row =[ticker,model_name,window_days, alpha, epochs, predict_days, batch_size,acc, prec, recall, f1]
        result_df.loc[len(result_df)] = new_row
        with open(model_file, 'wb') as file:
            pickle.dump(model, file)   
        result_df.to_csv(result_file, index=False)
        # Free memory
        del model
        tf.keras.backend.clear_session()
        gc.collect()

    trend_mapping = {"UP": 1, "DOWN": 2, "SIDEWAY": 3}
    em_y = fulldata['EMStrend'].dropna().to_list()
    lb_y = fulldata['Labels'].dropna().to_list()
    emyy = [trend_mapping[em] for em in em_y]
    lbyy = [trend_mapping[em] for em in lb_y]

    acc, prec, recall, f1 = cal_accuracy(ticker, lbyy, emyy, model_name)
    new_row =[ticker,'4MAs',window_days, alpha, epochs, predict_days, batch_size,acc, prec, recall, f1]
    result_df.loc[len(result_df)] = new_row
    result_df.to_csv(result_file, index=False)
    return result_df

In [4]:
tickers = ['^HSI','SPY','NVDA','TSLA']
window_days_l = [25, 50, 75]
alpha_l = [0.4, 0.5, 0.6]
epochs_l = [25,50]
predict_days_l = [5,8]
batch_size = 32

In [5]:
column_names=['ticker','model','window_days','alpha','epochs','predict_days','batch_size','accuracy','precision','recall','f1']

if os.path.exists(result_file):
    print(f"Reload {result_file} for append result.")
    result = pd.read_csv(result_file)
else:
    print(f"Create {result_file}")
    result=pd.DataFrame(columns=column_names)


Reload 4MAs_result.csv for append result.


In [6]:
# for ticker in tickers:
#     for window_days in window_days_l:
#         for alpha in alpha_l:
#             for epochs in epochs_l:
#                 for predict_days in predict_days_l:
#                     print( window_days, alpha, epochs, predict_days)
#                     result = main(ticker, window_days, alpha, epochs, predict_days, batch_size, result)

In [7]:
display(result)

,ticker,model,window_days,alpha,epochs,predict_days,batch_size,accuracy,precision,recall,f1
0,^HSI,CNN,25,0.4,25,5,32,0.784946,0.757092,0.784946,0.769622
1,^HSI,CNN-LSTM,25,0.4,25,5,32,0.440860,0.194358,0.440860,0.269780
2,^HSI,Dense,25,0.4,25,5,32,0.440860,0.194358,0.440860,0.269780
3,^HSI,4MAs,25,0.4,25,5,32,0.083871,0.050065,0.083871,0.053872
4,^HSI,CNN,25,0.4,25,8,32,0.752688,0.753147,0.752688,0.752815
5,^HSI,CNN-LSTM,25,0.4,25,8,32,0.462366,0.213782,0.462366,0.292378
6,^HSI,Dense,25,0.4,25,8,32,0.462366,0.213782,0.462366,0.292378
7,^HSI,4MAs,25,0.4,25,8,32,0.079570,0.052641,0.079570,0.053105
8,^HSI,CNN,25,0.4,50,5,32,0.795699,0.767337,0.795699,0.779814
9,^HSI,CNN-LSTM,25,0.4,50,5,32,0.440860,0.194358,0.440860,0.269780


In [8]:
ticker = '^HSI'
for window_days in window_days_l:
    for alpha in alpha_l:
        for epochs in epochs_l:
            for predict_days in predict_days_l:
                print( window_days, alpha, epochs, predict_days)
                result = main(ticker, window_days, alpha, epochs, predict_days, batch_size, result)
                break

25 0.4 25 5
models/^HSI_25_0.4_25_5_32.pkl model exist
25 0.4 50 5
models/^HSI_25_0.4_50_5_32.pkl model exist
25 0.5 25 5
models/^HSI_25_0.5_25_5_32.pkl model exist
25 0.5 50 5
Generating candlestick charts for ^HSI with gap=5, days=25, alpha=0.5, train_name=^HSI_25_0.5_50_5_32, test_name=^HSI_25_0.5_50_5_32
YF.download() has changed argument auto_adjust default to True
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 491 entries, 2023-06-20 to 2025-06-20
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   491 non-null    float64
 1   High    491 non-null    float64
 2   Low     491 non-null    float64
 3   Open    491 non-null    float64
 4   Volume  491 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 23.0 KB
None
Close on [ 2023-07-26 00:00:00 ]= 19365.140625 [ 2023-07-27 00:00:00 ]= 19172.05078125
Close on [ 2023-07-27 00:00:00 ]= 19639.109375 [ 2023-07-28 00:00:00 ]= 18934.359375
Close on [ 2023-0

/home/thomas/projects/Fin-Agents/venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1750525678.214499  479254 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1750525678.238737  479254 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1750525678.238791  479254 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1750525678.240600  479


Training CNN...
Epoch 1/50


I0000 00:00:1750525679.807016  479531 service.cc:146] XLA service 0x71f73c004500 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750525679.807062  479531 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-06-21 10:07:59.822423: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-06-21 10:07:59.910340: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 5/10 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.4531 - loss: 7.1000

...


Training CNN...
Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 371ms/step - accuracy: 0.3694 - loss: 8.1015 - val_accuracy: 0.4789 - val_loss: 1.1781
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5617 - loss: 1.0945 - val_accuracy: 0.8310 - val_loss: 0.5601
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.8449 - loss: 0.4771 - val_accuracy: 0.8451 - val_loss: 0.4439
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.8755 - loss: 0.3420 - val_accuracy: 0.8451 - val_loss: 0.3452
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9123 - loss: 0.2482 - val_accuracy: 0.8451 - val_loss: 0.3638
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9055 - loss: 0.2027 - val_accuracy: 0.8592 - val_loss: 0.3873
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9743 - loss: 0.1487 - val_accuracy: 0.8592 - val_loss: 0.4191
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9760 - loss: 0.0871 - val_accuracy: 0.8028 -

/home/thomas/projects/Fin-Agents/venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/thomas/projects/Fin-Agents/venv/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Training CNN...
Epoch 1/25


2025-06-21 10:23:53.056905: W external/local_tsl/tsl/framework/bfc_allocator.cc:482] Allocator (GPU_0_bfc) ran out of memory trying to allocate 651.47MiB (rounded to 683118592)requested by op 
2025-06-21 10:23:53.057196: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2025-06-21 10:23:53.057241: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 8785, Chunks in use: 8785. 2.14MiB allocated for chunks. 2.14MiB in use in bin. 50.0KiB client-requested in use in bin.
2025-06-21 10:23:53.057254: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 58, Chunks in use: 58. 31.2KiB allocated for chunks. 31.2KiB in use in bin. 29.0KiB client-requested in use in bin.
2025-06-21 10:23:53.057261: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (1024): 	Total Chunks: 93, Chunks in use: 93. 123.2KiB allocated for chunks. 123.2KiB in use in bin. 120.0KiB client-requested in use in bi

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/thomas/projects/Fin-Agents/venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/thomas/projects/Fin-Agents/venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/thomas/projects/Fin-Agents/venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/thomas/projects/Fin-Agents/venv/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/ho

In [ ]:
display(result)